In [ ]:
%%bash

# 1. Limpieza y Preparación de Datos
rm -f mlp_cuda main.cu # Borramos ejecutables viejos
mkdir -p data          # Creamos la carpeta data

# Descargar MNIST si no existe
if [ ! -f data/train-images-idx3-ubyte ]; then
    echo "Descargando dataset MNIST..."
    wget -q -O data/train-images-idx3-ubyte.gz https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz
    wget -q -O data/train-labels-idx1-ubyte.gz https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz
    gunzip -f data/*.gz
fi

# 2. Crear el archivo main.cu
cat <<EOF > main.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Incluimos directamente los headers y fuentes para facilitar compilación en Colab
#include "mnist_loader.h"
#include "mnist_loader.c"
#include "mlp_cuda.h"

// RUTAS AJUSTADAS PARA COLAB (Carpeta data/ local)
#define TRAIN_IMG "data/train-images-idx3-ubyte"
#define TRAIN_LBL "data/train-labels-idx1-ubyte"

int main() {
    printf("--- Escenario 3: GPU CUDA Nativo (C++) ---\n");

    // Verificar GPU
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    if (deviceCount == 0) {
        printf("ERROR: No se detectó GPU CUDA. Verifica Entorno de Ejecución > T4 GPU\n");
        return 1;
    }

    // Cargar datos
    int num_train, r, c;
    float* X_raw = load_mnist_images(TRAIN_IMG, &num_train, &r, &c);

    if (!X_raw) {
        printf("ERROR CRÍTICO: No se encontró el archivo de datos en %s\n", TRAIN_IMG);
        return 1;
    }
    printf("Datos cargados correctamente: %d imágenes.\n", num_train);

    // Transponer datos en CPU (Optimización de acceso a memoria)
    // Convertimos de (N x 784) a (784 x N) para que la GPU lea linealmente
    printf("Transponiendo datos en RAM...\n");
    float* X_T = (float*)malloc(num_train * 784 * sizeof(float));
    for(int i=0; i<num_train; i++) {
        for(int j=0; j<784; j++) {
            X_T[j * num_train + i] = X_raw[i * 784 + j];
        }
    }

    MLPCuda net;
    // Inicializar Red: 784 entrada -> 512 oculta -> 10 salida, LR=0.1
    mlp_init(&net, 784, 512, 10, 0.1f);

    // Entrenar
    // X_T ya está transpuesto, pasamos NULL en labels porque este benchmark es de velocidad Forward
    // Usaremos Batch Size 64 y 10 Epochs
    mlp_train(&net, X_T, NULL, num_train, 10, 64);

    // Limpieza
    mlp_free(&net);
    free(X_raw);
    free(X_T);
    return 0;
}
EOF

# 3. Compilar y Ejecutar
echo "Compilando..."
nvcc -o mlp_cuda main.cu mlp_cuda.cu

echo "Ejecutando..."
./mlp_cuda

Compilando...
Ejecutando...
--- Escenario 3: GPU CUDA Nativo (C++) ---
Datos cargados correctamente: 60000 imágenes.
Transponiendo datos en RAM...
Iniciando entrenamiento CUDA...

>>> Tiempo Total GPU (C++ CUDA): 0.65 segundos <<<
